# fishfASE

A multi-omics approach analyzing allele specific expression (ASE) to identify cis-regulatory variation influecing rapidly evolving craniofacial phenotypes in Cyprinodon pupfishes.

This notebook is used to assign maternal and paternal alleles inherited in F1 hybrids.

Prior steps involve require SNPs calling from whole-genome seuqneces in F0 parents. [See here](https://github.com/joemcgirr/fishfASE/tree/master/genomes).

The input is a data frame consisting of biallelic SNPs output by GATK. The SNPs are called from RNAseq reads belonging to F1 hybrids between three young species of Cyprinodon pupfishes. Sites that are alternatively homozygous in the two parents used to generate F1 hybrids should be heterozygous in hybrids. These scripts were used for two related publications:

McGirr JA and Martin CH. 2020. Few fixed variants between trophic specialist pupfish species reveal candidate cis-regulatory alleles underlying rapid craniofacial divergence. [Molecular Biology and Evolution pdf](https://github.com/joemcgirr/joemcgirr.github.io/blob/master/files/papers/mcgirr_2020.pdf)

McGirr JA and Martin CH. 2020. Ecological divergence in sympatry causes gene misexpression in hybrids. [Molecular Ecology pdf](https://github.com/joemcgirr/joemcgirr.github.io/blob/master/files/papers/mcgirr_2020b.pdf)


## Load Libraries

In [4]:
import io
import pandas as pd
import numpy as np
import math

## Functions to generate slurm script headers

In [5]:
def sbatch_header(job,mem,tasks,hours):
    #sbatch submission script header
    script = 'script_' + job + '.sh'
    outfile = io.open(script,'w', newline='\n')    
    outfile.write('#!/bin/bash\n\n#SBATCH --job-name='+job+'\n')
    outfile.write('#SBATCH --mem='+mem+'G \n')
    outfile.write('#SBATCH --ntasks='+tasks+' \n')
    outfile.write('#SBATCH -e '+job+'_%A_%a.err \n')
    outfile.write('#SBATCH --time='+hours+':00:00  \n')
    outfile.write('#SBATCH --mail-user=jamcgirr@ucdavis.edu ##email you when job starts,ends,etc\n#SBATCH --mail-type=ALL\n')
    outfile.write('#SBATCH -p high \n\n')
    outfile.close()
    
def sbatch_header_loop(job,mem,tasks,hours,infile):
    #sbatch submission script header
    script = 'script_' + infile + job + '.sh'
    outfile = io.open(script,'w', newline='\n') 
    jobname= infile + job   
    outfile.write('#!/bin/bash\n\n#SBATCH --job-name='+jobname+'\n')
    outfile.write('#SBATCH --mem='+mem+'G \n')
    outfile.write('#SBATCH --ntasks='+tasks+' \n')
    outfile.write('#SBATCH -e '+jobname+'_%A_%a.err \n')
    outfile.write('#SBATCH --time='+hours+':00:00 \n')
    outfile.write('#SBATCH --mail-user=jamcgirr@ucdavis.edu ##email you when job starts,ends,etc\n#SBATCH --mail-type=ALL\n')
    outfile.write('#SBATCH -p high \n\n')
    outfile.close()

## Output allele counts with GATK

Use GATK to generate SNP table from phased `.vcf` for each individual

Input = sorted `.bam` files
Output = SNP table with alleles counts

In [6]:
job_name = '_allele_counts'
bams_dir = '/pine/scr/j/m/jmcgirr/pupfish_transcriptomes/wasp/final_unbiased_bams/'
counts_dir = '/pine/scr/j/m/jmcgirr/pupfish_transcriptomes/ase/allele_counts/'
rg_bams_dir = '/pine/scr/j/m/jmcgirr/pupfish_transcriptomes/ase/bams_for_phasing/'

# sample names
infiles = ["CAE1","CAE2","CAE3","CAE4","CAE5","CAT1","CAT2","CAT3","CME1","CME2","CME5","CMT1","CMT2","CMT3","CPAE1","CPAE2","CPAE3","CPAJ1","CPAJ2","CPAJ3","CPE1","CPE2","CPE3","CPE4","CPE5","CPME1","CPME2","CPME3","CPMJ1","CPMJ2","CPMJ3","CPPE1","CPPE2","CPPE3","CPPJ1","CPPJ2","CPT1","CPT2","CPT3","CPU1","CPU3","CPU5","CQE1","CQE2","CQE3","CQT1","CQT2","CUT1","CUT2","CUT3","CVE1","CVE2","CVE5","CVT1","CVT2","CVT3","CWE2","CWE3","CWE4","CWT1","CWT2","CWT3","CXE2","CXE3","CXE4","CXT1","CXT2","CXT3","LFE2","LFE3","LFE4","LFT1","LFT2","LFT3","LGE3","LGE4","LGE5","LGT1","LGT2","LGT3","LIE2","LIE3","LIE5","LIT1","LIT2","LIT3","LKE1","LKE2","LKE3","LKT1","LKT2","LKT3","LLAE1","LLAE2","LLAE3","LLAJ1","LLAJ2","LLAJ3","LLME1","LLME2","LLME3","LLMJ1","LLMJ2","LLMJ3","LPE1","LPE2","LPE3","LPE4","LPE5","NAE1","NAE2","NAE4","NAT1","NAT2","NAT3","NCE1","NCE2","NCE3","NCE4","NCE5","NCT1","NCT2","NCT3","OAE1","OAE2","OAE3","OAE4","OAE5","OAT1","OAT2","OAT3","OME1","OME2","OME3","OME4","OME5","OMT1","OMT2","OMT3","OPE1","OPE2","OPE3","OPE4","OPE5","OPT1","OPT2","OPT3","ORE1","ORE2","ORE3","ORE4","ORE5","ORT1","ORT2","ORT3","OUE1","OUE3","OUE4","OUT1","OUT2","OUT3","OVE1","OVE4","OVE5","OVT1","OVT2","OVT3","OXE2","OXT1","OXT2","OXT3","OYE1","OYE2","OYE3","OYE4","OYE5","OYT1","OYT2","OYT3","OZE2","OZE4","OZE5","OZT1","OZT2","OZT3","PAE1","PAE2","PAE5","PAT1","PAT2","PAT3"]

for infile in infiles:
    script = 'script_' + infile + job_name + '.sh'
    sbatch_header_loop(job_name,'32','4','24', infile)
    o = io.open(script,'a+', newline='\n')
    
    # sbatch command
    
    o.write('module load java \n')
    o.write('java -jar /proj/cmarlab/users/joe/picard.jar AddOrReplaceReadGroups I='+bams_dir+infile+'_filtered.merged.sort.bam O='+rg_bams_dir+infile+'_filtered.merged.sort.RG.bam RGLB=lib1 RGPL=illumina RGPU=unit1 RGSM='+infile+' \n')
    o.write('module load samtools \n')
    o.write('samtools index '+rg_bams_dir+infile+'_filtered.merged.sort.RG.bam \n')
    o.write('module load gatk/3.8-0 \n')
    o.write('gatk -T SplitNCigarReads -drf DuplicateRead -rf ReassignOneMappingQuality -U ALLOW_N_CIGAR_READS -R /proj/cmarlab/users/joe/Cyprinodon/bronto/asm.racon.fasta -I '+rg_bams_dir+infile+'_filtered.merged.sort.RG.bam -o '+rg_bams_dir+infile+'_filtered.merged.sort.RG.split.bam \n')
    o.write('module load samtools \n')
    o.write('samtools index '+rg_bams_dir+infile+'_filtered.merged.sort.RG.split.bam \n')
    o.write('gatk -T ReadBackedPhasing -R /proj/cmarlab/users/joe/Cyprinodon/bronto/asm.racon.fasta -I '+rg_bams_dir+infile+'_filtered.merged.sort.RG.split.bam --variant /pine/scr/j/m/jmcgirr/pupfish_transcriptomes/vcf/all_rna_wasp_unbiased_filtered_snps_final.vcf -o /pine/scr/j/m/jmcgirr/pupfish_transcriptomes/wasp/vcf/'+infile+'_wasp_unbiased_phased.vcf --phaseQualityThresh 20.0 \n')
    o.write('gatk -T ASEReadCounter -R /proj/cmarlab/users/joe/Cyprinodon/bronto/asm.racon.fasta -U ALLOW_SEQ_DICT_INCOMPATIBILITY -o '+counts_dir+infile+'_counts.csv -I '+rg_bams_dir+infile+'_filtered.merged.sort.RG.split.bam -sites /pine/scr/j/m/jmcgirr/pupfish_transcriptomes/wasp/vcf/'+infile+'_wasp_unbiased_phased.vcf \n')           
    o.write('gatk -T VariantsToTable -R /proj/cmarlab/users/joe/Cyprinodon/bronto/asm.racon.fasta -V /pine/scr/j/m/jmcgirr/pupfish_transcriptomes/wasp/vcf/'+infile+'_wasp_unbiased_phased.vcf -F CHROM -F POS -GF GT -GF HP -o '+counts_dir+infile+'_snp_table.txt')


    #run sbatch submission
    o.write('\n\n#run: sbatch '+script)
    o.close()
    

## Assign parental alleles

Create dictionaries matching F1 offspring samples names to their parents sample names. Output heterozygous sites and the read counts for alleles contributed by each parent.

In [ ]:
# dictionary to match rna samples with mother dna samples
mom_dict = {"CAE1": "CRPA1000", "CAE2": "CRPA1000", "CAE3": "CRPA1000", "CAE4": "CRPA1000", "CAE5": "CRPA1000", "CAT1": "CRPA1000", "CAT2": "CRPA1000", "CAT3": "CRPA1000", "CME1": "CRPM1000", "CME2": "CRPM1000", "CME5": "CRPM1000", "CMT1": "CRPM1000", "CMT2": "CRPM1000", "CMT3": "CRPM1000", "CPE1": "CRPP1000", "CPE2": "CRPP1000", "CPE3": "CRPP1000", "CPE4": "CRPP1000", "CPE5": "CRPP1000", "CPT1": "CRPP1000", "CPT2": "CRPP1000", "CPT3": "CRPP1000", "CPU1": "CRPA1000", "CPU3": "CRPA1000", "CPU5": "CRPA1000", "CQE1": "CUNP10", "CQE2": "CUNP10", "CQE3": "CUNP10", "CQT1": "CUNP10", "CQT2": "CUNP10", "CUT1": "CRPA1000", "CUT2": "CRPA1000", "CUT3": "CRPA1000", "CVE1": "CRPA1000", "CVE2": "CRPA1000", "CVE5": "CRPA1000", "CVT1": "CRPA1000", "CVT2": "CRPA1000", "CVT3": "CRPA1000", "CWE2": "CRPM1000", "CWE3": "CRPM1000", "CWE4": "CRPM1000", "CWT1": "CRPM1000", "CWT2": "CRPM1000", "CWT3": "CRPM1000", "CXE2": "CRPM1000", "CXE3": "CRPM1000", "CXE4": "CRPM1000", "CXT1": "CRPM1000", "CXT2": "CRPM1000", "CXT3": "CRPM1000", "LFE2": "CRPA1000", "LFE3": "CRPA1000", "LFE4": "CRPA1000", "LFT1": "CRPA1000", "LFT2": "CRPA1000", "LFT3": "CRPA1000", "LGE3": "CRPM1000", "LGE4": "CRPM1000", "LGT1": "CRPM1000", "LGT2": "CRPM1000", "LGT3": "CRPM1000", "LIE2": "OSPA1000", "LIE3": "OSPA1000", "LIE5": "OSPA1000", "LIT1": "OSPA1000", "LIT2": "OSPA1000", "LIT3": "OSPA1000", "LKE1": "OSPP1000", "LKE2": "OSPP1000", "LKE3": "OSPP1000", "LKT1": "OSPP1000", "LKT2": "OSPP1000", "LKT3": "OSPP1000", "NAE1": "NCCA1000", "NAE2": "NCCA1000", "NAE4": "NCCA1000", "NAT1": "NCCA1000", "NAT2": "NCCA1000", "NAT3": "NCCA1000", "NCE1": "NCCA1000", "NCE2": "NCCA1000", "NCE3": "NCCA1000", "NCE4": "NCCA1000", "NCE5": "NCCA1000", "NCT1": "NCCA1000", "NCT2": "NCCA1000", "NCT3": "NCCA1000", "OAE1": "OSPA1000", "OAE2": "OSPA1000", "OAE3": "OSPA1000", "OAE4": "OSPA1000", "OAT1": "OSPA1000", "OAT2": "OSPA1000", "OAT3": "OSPA1000", "OME1": "OSPM1000", "OME2": "OSPM1000", "OME3": "OSPM1000", "OME4": "OSPM1000", "OME5": "OSPM1000", "OMT1": "OSPM1000", "OMT2": "OSPM1000", "OMT3": "OSPM1000", "OPE1": "OSPP1000", "OPE2": "OSPP1000", "OPE3": "OSPP1000", "OPE4": "OSPP1000", "OPE5": "OSPP1000", "OPT1": "OSPP1000", "OPT2": "OSPP1000", "OPT3": "OSPP1000", "OUE1": "OSPA1000", "OUE3": "OSPA1000", "OUE4": "OSPA1000", "OUT1": "OSPA1000", "OUT2": "OSPA1000", "OUT3": "OSPA1000", "OVE1": "OSPA1000", "OVE4": "OSPA1000", "OVE5": "OSPA1000", "OVT1": "OSPA1000", "OVT2": "OSPA1000", "OVT3": "OSPA1000", "OXE2": "OSPM1000", "OXT1": "OSPM1000", "OXT2": "OSPM1000", "OXT3": "OSPM1000", "OYE1": "OSPP1000", "OYE2": "OSPP1000", "OYE3": "OSPP1000", "OYE4": "OSPP1000", "OYE5": "OSPP1000", "OYT1": "OSPP1000", "OYT2": "OSPP1000", "OYT3": "OSPP1000", "OZE2": "OSPP1000", "OZE4": "OSPP1000", "OZE5": "OSPP1000", "OZT1": "OSPP1000", "OZT2": "OSPP1000", "OZT3": "OSPP1000", "PAE1": "CUNP10", "PAE2": "CUNP10", "PAE5": "CUNP10", "PAT1": "CUNP10", "PAT2": "CUNP10", "PAT3": "CUNP10"}

# dad double check to confirm that alleles are alternatively homozygous in mom AND dad
infiles = ["CPU1","CPU3","CPU5","CUT1","CUT2","CUT3","CVE1","CVE2","CVE5","CVT1","CVT2","CVT3","CWE2","CWE3","CWE4","CWT1","CWT2","CWT3","CXE2","CXE3","CXE4","CXT1","CXT2","CXT3","LFE2","LFE3","LFE4","LFT1","LFT2","LFT3","LGE3","LGE4","LGT1","LGT2","LGT3","LIE2","LIE3","LIE5","LIT1","LIT2","LIT3","LKE1","LKE2","LKE3","LKT1","LKT2","LKT3","NAE1","NAE2","NAE4","NAT1","NAT2","NAT3","OUE1","OUE3","OUE4","OUT1","OUT2","OUT3","OVE1","OVE4","OVE5","OVT1","OVT2","OVT3","OXE2","OXT1","OXT2","OXT3","OYE1","OYE2","OYE3","OYE4","OYE5","OYT1","OYT2","OYT3","OZE2","OZE4","OZE5","OZT1","OZT2","OZT3","PAE1","PAE2","PAE5","PAT1","PAT2","PAT3"]
dad_dict = {"CPU1": "CRPM1001","CPU3": "CRPM1001","CPU5": "CRPM1001","CUT1": "CRPM1001","CUT2": "CRPM1001","CUT3": "CRPM1001","CVE1": "CRPP1001","CVE2": "CRPP1001","CVE5": "CRPP1001","CVT1": "CRPP1001","CVT2": "CRPP1001","CVT3": "CRPP1001","CWE2": "CRPP1001","CWE3": "CRPP1001","CWE4": "CRPP1001","CWT1": "CRPP1001","CWT2": "CRPP1001","CWT3": "CRPP1001","CXE2": "CRPA1003","CXE3": "CRPA1003","CXE4": "CRPA1003","CXT1": "CRPA1003","CXT2": "CRPA1003","CXT3": "CRPA1003","LFE2": "OSPA1001","LFE3": "OSPA1001","LFE4": "OSPA1001","LFT1": "OSPA1001","LFT2": "OSPA1001","LFT3": "OSPA1001","LGE3": "OSPM1001","LGE4": "OSPM1001","LGT1": "OSPM1001","LGT2": "OSPM1001","LGT3": "OSPM1001","LIE2": "CRPA1001","LIE3": "CRPA1001","LIE5": "CRPA1001","LIT1": "CRPA1001","LIT2": "CRPA1001","LIT3": "CRPA1001","LKE1": "CRPP1001","LKE2": "CRPP1001","LKE3": "CRPP1001","LKT1": "CRPP1001","LKT2": "CRPP1001","LKT3": "CRPP1001","NAE1": "CRPA1001","NAE2": "CRPA1001","NAE4": "CRPA1001","NAT1": "CRPA1001","NAT2": "CRPA1001","NAT3": "CRPA1001","OUE1": "OSPM1001","OUE3": "OSPM1001","OUE4": "OSPM1001","OUT1": "OSPM1001","OUT2": "OSPM1001","OUT3": "OSPM1001","OVE1": "OSPP1001","OVE4": "OSPP1001","OVE5": "OSPP1001","OVT1": "OSPP1001","OVT2": "OSPP1001","OVT3": "OSPP1001","OXE2": "OSPA1001","OXT1": "OSPA1001","OXT2": "OSPA1001","OXT3": "OSPA1001","OYE1": "OSPA1001","OYE2": "OSPA1001","OYE3": "OSPA1001","OYE4": "OSPA1001","OYE5": "OSPA1001","OYT1": "OSPA1001","OYT2": "OSPA1001","OYT3": "OSPA1001","OZE2": "OSPM1001","OZE4": "OSPM1001","OZE5": "OSPM1001","OZT1": "OSPM1001","OZT2": "OSPM1001","OZT3": "OSPM1001","PAE1": "CRPA1001","PAE2": "CRPA1001","PAE5": "CRPA1001","PAT1": "CRPA1001","PAT2": "CRPA1001","PAT3": "CRPA1001"}

cts_dir = 'C:/Users/jmcgirr/Documents/remote_pups/ase/allele_counts/'
snp_dir ='C:/Users/jmcgirr/Documents/remote_pups/ase/'
out_dir = 'C:/Users/jmcgirr/Documents/remote_pups/ase/parental_counts/dad_doublecheck/'
all_rna_snps = pd.read_csv(snp_dir + "rna_snp_table.txt", sep ='\t')
all_dna_snps = pd.read_csv(snp_dir + "rna_parents_filtered_snps_passed.Q20.MAF0.05.MAXMISS0.5.recode_snp_table.txt", sep ='\t')
all_dna_snps['snpIndex'] = all_dna_snps['CHROM'].astype(str) + ':' + all_dna_snps['POS'].astype(str)

for infile in infiles:

    cts = pd.read_csv(cts_dir +infile + "_counts.csv", sep = '\t')
    cts['snpIndex'] = cts['contig'].astype(str) + ':'+ cts['position'].astype(str) 
    cts = cts[['snpIndex','refAllele','altAllele','refCount', 'altCount', 'totalCount']]
    mom = mom_dict[infile] +".GT"
    snps = all_dna_snps[[mom,'snpIndex']]
    
    mom_kid = cts.merge(snps, on='snpIndex')
    mom_kid = mom_kid.join(mom_kid[mom].str.split('/', 1, expand=True).rename(columns={0:'momAllele', 1:'a2'}))
    
    # only analyze homozygous alleles in maternal samples that are heterozygous in offspring
    mom_kid = mom_kid[mom_kid['momAllele'] == mom_kid['a2']]
    mom_kid = mom_kid[(mom_kid['momAllele'] == mom_kid['refAllele']) | (mom_kid['momAllele'] == mom_kid['altAllele'])]
    
    # set minimum coverage at site (>= 10 counts at each site)
    mom_kid = mom_kid[(mom_kid['refCount'] >= 10) & (mom_kid['altCount'] >= 10)]
    mom_kid.loc[mom_kid['refAllele'] == mom_kid['momAllele'], 'momCount'] = mom_kid['refCount']
    mom_kid.loc[mom_kid['altAllele'] == mom_kid['momAllele'], 'momCount'] = mom_kid['altCount']
    mom_kid = mom_kid.join(mom_kid['snpIndex'].str.split(':', 1, expand=True).rename(columns={0:'chrom', 1:'position'}))
    mom_kid = mom_kid[['chrom','position','snpIndex','refAllele','altAllele','refCount','altCount','totalCount','momAllele', 'momCount']]
    
    mom_kid['dadCount'] = mom_kid['totalCount'] - mom_kid['momCount']
    mom_kid['momAllele_is_refAllele'] = np.where(mom_kid['refAllele'] == mom_kid['momAllele'], 'yes', 'no')
    mom_kid['momAllele_is_majorAllele'] = np.where(mom_kid['momCount'] > mom_kid['dadCount'], 'yes', 'no')
    
    # confirm that the paternal allele is different from maternal allele
    dad = dad_dict[infile] +".GT"
    snps_dad = all_dna_snps[[dad,'snpIndex']]
    snps_dad = snps_dad.join(snps_dad[dad].str.split('/', 1, expand=True).rename(columns={0:'dadAllelereal1', 1:'dadAllelereal2'}))
    snps_dad = snps_dad[snps_dad['dadAllelereal1'] == snps_dad['dadAllelereal2']]
    mom_kid = mom_kid.merge(snps_dad, on='snpIndex')
    mom_kid = mom_kid[(mom_kid['momAllele'] != mom_kid['dadAllelereal1']) & (mom_kid['momAllele'] != mom_kid['dadAllelereal2'])]

    
    mom_kid.to_csv(out_dir +infile+'_parental_counts.txt',index=False, sep = "\t")


## Use parental allele counts to estimate allele specific expression in R with MBASED

See [mbased.R](https://github.com/joemcgirr/fishfASE/blob/master/ase/mbased.R)